# Mochila Quadrática $0$-$1$ (QKP)

## Dados

Considere dos seguintes dados:

-  $N = \{1, \ldots, n\}$ um conjunto dos objetos.

-  $P = \{p_{ij} \in \mathbb{R}_{+} : \ i \leq j, \ i, j \in N \}$ o conjunto dos benefícios aos itens de $N$ e a relação entre eles.

-  $W = \{ w_i \in \mathbb{N} : i \in N \}$ o conjunto dos pesos dos itens de $N$.

-  $c \in \mathbb{N}$ capacidade da mochila, onde $\displaystyle\max_{i \in N} w_i \leq c < \sum_{i  \in N} w_i$.

## QKP 0-1

Utilizando os dados acima para definir o problema da mochila quadrática 0-1.

\begin{align}
max \; & \sum_{i \in N} q_{i} x_i + \sum_{i = 1}^{n-1} \sum_{j =i+1}^{n}
p_{ij} x_{i} x_{j} \\
s. a  \; & \sum_{i \in N} w_{i} x_{i} \leq c, \\
& x_{i} \in \left\{0, 1 \right\}, \ \ i  \in N.
\end{align}

onde $q_{i} = p_{ii}, \ i \in N$.

## Interpretação em Teoria do Grafos

Seja $G=(V,E)$ um grafo completo não direcionado com $n$ vértices onde cada vértice $i \in V$ possui um benefício $q_{i}$ e tem um peso $w_i$. 

Da mesma forma, cada aresta $e=(i,j) \in E$ possui um benefício $p_{ij}$. 

Definido $G$, pede-se para encontrar um subconjunto de vértices $S \subseteq V$ onde $\sum_{i \in S} w_i \leq c$ e o benefício total auferido seja máximo.



## Modelando o problema e resolvendo com gurobipy

In [16]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

In [17]:
def read_instance(dim,perc,t):

    instance = f"instances/qkp/{dim}/{dim}_{perc}_{t}.txt"

    with open(instance, 'r') as file: linhas = file.readlines()

    # remove linha vazia inicial e elimina os "\n" de cada linha
    linhas = [a.strip() for a in linhas] 

    # ler o tamanho da instancia
    n = int(linhas[0]) 

    # ler a diagonal da matriz
    d = np.fromstring(linhas[1], dtype=int, sep = ' ') 

    # define a matriz
    p = np.zeros((n,n), dtype=int) 

    # preenche a diagonal
    for i in range(n): 
        p[i][i] = d[i]

    # preenche o resto da matriz
    for i in range(n-1): 
        linha = np.fromstring(linhas[i+2], dtype=int, sep = ' ')
        for j in range(n-(i+1)):
            p[i][j+i+1] = linha[j]
            p[j+i+1][i] = p[i][j+i+1]

    # ler a capacidade
    c = int(linhas[n+2]) 

    # ler os pesos
    w = np.fromstring(linhas[n+3], dtype=int, sep = ' ') 

    return n, p, w, c

In [18]:
def qknapsack(dim,perc,t):

    n, p, w, c = read_instance(dim,perc,t)

    #cria o modelo
    model = gp.Model("qkp") 

    x = []
    for j in range(0, n):
        x.append(model.addVar(vtype=GRB.BINARY, name="x_{}".format(j+1)))
        
    model.Params.TimeLimit = 120
    model.Params.MIPGap = 1.e-6
    model.Params.Threads = 1
    
    # Turn off display
    gp.setParam('OutputFlag', 0)

    obj = 0 
    for i in range(0, n):
        obj += p[i][i] * x[i]
        for j in range(i+1, n):
            obj += 2*p[i][j] * x[i] * x[j]

    model.setObjective(obj, GRB.MAXIMIZE)

    constr = 0
    for j in range(0, n):
        constr += (w[j] * x[j])
    model.addConstr(constr <= c)

    model.write(f"lp/qkp_{dim}_{perc}_{t}.lp")

    model.optimize()
    
    status = 0
    if model.status == GRB.OPTIMAL:
        status = 1
 
    ub = model.objBound
    lb = model.objVal
    gap = model.MIPGap
    time = model.Runtime
    nodes = model.NodeCount

    model.dispose()

    #lower bound, upper bound, gap, time, nodes
    arquivo = open(f'result/qkp_{dim}_{perc}.csv','a')
    arquivo.write(
        str(f"{dim}_{perc}_{t}")+';'
        +str(round(lb,1))+';'
        +str(round(ub,1))+';'
        +str(round(gap,2))+';'
        +str(round(time,2))+';'
        +str(round(nodes,1))+';'
        +str(round(status,1))+'\n'
    )
    arquivo.close()

In [22]:
# __main__ : “top-level code environment”

# What is the “top-level code environment”?
# __main__ is the name of the environment where top-level code is run. 
# “Top-level code” is the first user-specified Python module that starts running. 
# It’s “top-level” because it imports all other modules that the program needs. 
# Sometimes “top-level code” is called an entry point to the application.

if __name__ == "__main__":

    for dim in [50]:
        for perc in [75]:
            for t in range(1,6):
                qknapsack(dim,perc,t)

## Tabelando os resultados 

In [23]:
dim = 50
perc = 75
inst_=f'result/qkp_{dim}_{perc}.csv'

df = pd.DataFrame()
df = pd.read_csv(inst_,header=None,sep=';')

tab = pd.DataFrame()
tab = pd.concat([tab, df], ignore_index=True)
tab.columns = ['instance','bs','bb','gap','time','nodes','opt']

resume = pd.DataFrame({
    'instance':f"resume",
    'bs':tab["bs"].mean(),
    'bb':tab["bb"].mean(),
    'gap':tab['gap'].mean(),
    'time':tab['time'].mean(),
    'nodes':tab['nodes'].mean(),
    'opt':tab['opt'].sum(),
     },index=[f"qkp"]
)

tab = pd.concat([tab, resume], ignore_index=True)

tab["bb"] = tab["bb"].round(2)
tab["bs"] = tab["bs"].round(2)
tab["gap"] = tab["gap"].round(2)
tab["time"] = tab["time"].round(2)
tab["nodes"] = tab["nodes"].round(2)
tab["opt"] = tab["opt"].round().astype('Int64')

#tab
print(
    tab[['instance','bs','bb','gap','time','nodes','opt']].
    to_latex(index=False,float_format="%.2f")
)

\begin{tabular}{lrrrrrr}
\toprule
instance &       bs &       bb &  gap &  time &  nodes &  opt \\
\midrule
 50\_75\_1 & 27601.00 & 27601.00 & 0.00 &  0.37 &  44.00 &    1 \\
 50\_75\_1 & 27601.00 & 27601.00 & 0.00 &  0.31 &  44.00 &    1 \\
 50\_75\_1 & 27601.00 & 27601.00 & 0.00 &  0.27 &  44.00 &    1 \\
 50\_75\_1 & 27601.00 & 27601.00 & 0.00 &  0.28 &  44.00 &    1 \\
 50\_75\_1 & 27601.00 & 27601.00 & 0.00 &  0.28 &  44.00 &    1 \\
 50\_75\_2 & 23764.00 & 23764.00 & 0.00 &  0.51 &  93.00 &    1 \\
 50\_75\_3 & 45785.00 & 45785.00 & 0.00 &  0.10 &   1.00 &    1 \\
 50\_75\_4 &  4339.00 &  4339.00 & 0.00 &  0.72 & 123.00 &    1 \\
 50\_75\_5 & 43436.00 & 43436.00 & 0.00 &  0.10 &   1.00 &    1 \\
  resume & 28369.89 & 28369.89 & 0.00 &  0.33 &  48.67 &    9 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_16195/2954734729.py:34: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  tab[['instance','bs','bb','gap','time','nodes','opt']].
